<a href="https://colab.research.google.com/github/sindhugovi/Sea-Debris-Classification/blob/main/aqua_avengers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


In [ ]:
!pip install tensorflow numpy matplotlib scikit-learn

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()  # This will open a file upload dialog

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sindhu294","key":"9b61226d93595fc89443f4f7343dee91"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle(1).json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle(1).json

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `mv kaggle(1).json ~/.kaggle/'
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `chmod 600 ~/.kaggle/kaggle(1).json'


In [ ]:
!kaggle datasets download -d mexwell/trashcan-1-0 --unzip -p /content/trash_dataset



Dataset URL: https://www.kaggle.com/datasets/mexwell/trashcan-1-0
License(s): Attribution 4.0 International (CC BY 4.0)
 99% 524M/527M [00:15<00:00, 42.3MB/s]
100% 527M/527M [00:15<00:00, 35.6MB/s]


In [ ]:
import os
dataset_path = "/content/trash_dataset"
print("Extracted Files:", os.listdir(dataset_path))

Extracted Files: ['dataset']


In [ ]:
import os
import shutil
import pandas as pd
from pycocotools.coco import COCO

In [ ]:
dataset_path = "/content/trash_dataset/dataset/material_version"  # Change this if needed
train_json = os.path.join(dataset_path, "instances_train_trashcan.json")
val_json = os.path.join(dataset_path, "instances_val_trashcan.json")
train_img_dir = os.path.join(dataset_path, "train")
val_img_dir = os.path.join(dataset_path, "val")

# Define new paths for structured dataset
train_output_dir = os.path.join(dataset_path, "train_classified")
val_output_dir = os.path.join(dataset_path, "val_classified")

In [ ]:
# Load COCO JSON files
coco_train = COCO(train_json)
coco_val = COCO(val_json)

# Extract class names from JSON
categories = coco_train.loadCats(coco_train.getCatIds())
class_names = {cat['id']: cat['name'] for cat in categories}
print("Classes:", class_names)

loading annotations into memory...
Done (t=0.31s)
creating index...
index created!
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
Classes: {1: 'rov', 2: 'plant', 3: 'animal_fish', 4: 'animal_starfish', 5: 'animal_shells', 6: 'animal_crab', 7: 'animal_eel', 8: 'animal_etc', 9: 'trash_etc', 10: 'trash_fabric', 11: 'trash_fishing_gear', 12: 'trash_metal', 13: 'trash_paper', 14: 'trash_plastic', 15: 'trash_rubber', 16: 'trash_wood'}


In [ ]:
def create_class_dirs(base_path, class_names):
    for class_name in class_names.values():
        os.makedirs(os.path.join(base_path, class_name), exist_ok=True)

create_class_dirs(train_output_dir, class_names)
create_class_dirs(val_output_dir, class_names)

In [ ]:
def move_images(coco, src_folder, dst_folder):
    for img_id in coco.getImgIds():
        img_info = coco.loadImgs(img_id)[0]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        annotations = coco.loadAnns(ann_ids)

        # Assign the image to the most frequent class in annotations
        class_count = {}
        for ann in annotations:
            class_id = ann['category_id']
            class_name = class_names[class_id]
            class_count[class_name] = class_count.get(class_name, 0) + 1

        if class_count:
            primary_class = max(class_count, key=class_count.get)
            src_path = os.path.join(src_folder, img_info['file_name'])
            dst_path = os.path.join(dst_folder, primary_class, img_info['file_name'])

            if os.path.exists(src_path):
                shutil.move(src_path, dst_path)
            else:
                print(f"Missing file: {src_path}")

# Move training and validation images
move_images(coco_train, train_img_dir, train_output_dir)
move_images(coco_val, val_img_dir, val_output_dir)

print("✅ Dataset restructuring completed successfully.")


✅ Dataset restructuring completed successfully.


In [ ]:
import os

train_output_dir = "/content/trash_dataset/dataset/material_version/train_classified"
val_output_dir = "/content/trash_dataset/dataset/material_version/val_classified"

print("Training dataset structure:")
for class_name in os.listdir(train_output_dir):
    class_path = os.path.join(train_output_dir, class_name)
    print(f"{class_name}: {len(os.listdir(class_path))} images")

print("\nValidation dataset structure:")
for class_name in os.listdir(val_output_dir):
    class_path = os.path.join(val_output_dir, class_name)
    print(f"{class_name}: {len(os.listdir(class_path))} images")


Training dataset structure:
rov: 1775 images
trash_fabric: 173 images
plant: 137 images
trash_rubber: 48 images
trash_metal: 649 images
animal_fish: 447 images
animal_eel: 106 images
trash_etc: 1129 images
trash_fishing_gear: 29 images
trash_paper: 103 images
trash_wood: 237 images
animal_starfish: 54 images
animal_shells: 66 images
animal_crab: 111 images
animal_etc: 62 images
trash_plastic: 882 images

Validation dataset structure:
rov: 312 images
trash_fabric: 37 images
plant: 31 images
trash_rubber: 18 images
trash_metal: 127 images
animal_fish: 71 images
animal_eel: 34 images
trash_etc: 211 images
trash_fishing_gear: 4 images
trash_paper: 20 images
trash_wood: 62 images
animal_starfish: 35 images
animal_shells: 28 images
animal_crab: 11 images
animal_etc: 20 images
trash_plastic: 183 images


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/trash_dataset/dataset/material_version/train_classified"
val_dir = "/content/trash_dataset/dataset/material_version/val_classified"
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32
img_size = (224, 224)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

print("✅ Data loading completed.")


Found 6008 images belonging to 16 classes.
Found 1204 images belonging to 16 classes.
✅ Data loading completed.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Load the InceptionResNetV2 model (pretrained on ImageNet)
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)  # Increased neurons for better learning
x = Dense(256, activation='relu')(x)
output_layer = Dense(train_generator.num_classes, activation='softmax')(x)

# Define the final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Freeze base model layers (initially)
for layer in base_model.layers:
    layer.trainable = False

# Compile model with Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor='val_accuracy', mode='max'),  # Save best model
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),  # Reduce LR on plateau
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)  # Stop early if no improvement
]

# Train the model
history = model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=callbacks)

print("✅ Model training completed.")

# Fine-tuning: Unfreeze some layers for better accuracy
for layer in base_model.layers[-50:]:  # Unfreeze last 50 layers
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train again for fine-tuning
history_finetune = model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=callbacks)

print("✅ Fine-tuning completed, best model saved as 'best_model.keras'.")


219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 129s 534ms/step - accuracy: 0.3859 - loss: 1.9741 - val_accuracy: 0.3846 - val_loss: 2.0149 - learning_rate: 1.0000e-04
Epoch 2/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 71s 376ms/step - accuracy: 0.5207 - loss: 1.5502 - val_accuracy: 0.3912 - val_loss: 1.9623 - learning_rate: 1.0000e-04
Epoch 3/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 70s 374ms/step - accuracy: 0.5659 - loss: 1.4064 - val_accuracy: 0.4468 - val_loss: 1.8660 - learning_rate: 1.0000e-04
Epoch 4/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 70s 374ms/step - accuracy: 0.5933 - loss: 1.2661 - val_accuracy: 0.4518 - val_loss: 1.8769 - learning_rate: 1.0000e-04
Epoch 5/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 70s 369ms/step - accuracy: 0.6233 - loss: 1.1906 - val_accuracy: 0.4012 - val_loss: 1.8975 - learning_rate: 1.0000e-04
Epoch 6/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 68s 363ms/step - accuracy: 0.6396 - loss: 1.1419 - val_accuracy: 0.4485 - val_loss: 1.8309 - learning_rate: 1.0000e-04
Epoch 7/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 71s 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Save the trained model to a specific location
model.save('/content/drive/MyDrive/best_model.keras')
print("✅ Model saved to Google Drive!")


✅ Model saved to Google Drive!


In [ ]:
# Load the saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/best_model.keras')


In [ ]:
import tensorflow as tf

# Load your model (replace the path with where you saved your model)
model_path = '/content/drive/MyDrive/best_model.keras'
model = tf.keras.models.load_model(model_path)

# Print model summary to confirm it's loaded correctly
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_203 (Conv2D)       │ (None, 111, 111, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_203   │ (None, 111, 111, 32)   │             96 │ conv2d_203[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_203            │ (None, 111, 111, 32)   │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_204 (Conv2D)       │ (None, 109, 109, 32)   │          9,216 │ activation_203[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_204   │ (None, 109, 109, 32)   │             96 │ conv2d_204[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_204            │ (None, 109, 109, 32)   │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_205 (Conv2D)       │ (None, 109, 109, 64)   │         18,432 │ activation_204[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_205   │ (None, 109, 109, 64)   │            192 │ conv2d_205[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_205            │ (None, 109, 109, 64)   │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 54, 54, 64)     │              0 │ activation_205[0][0]   │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_206 (Conv2D)       │ (None, 54, 54, 80)     │          5,120 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_206   │ (None, 54, 54, 80)     │            240 │ conv2d_206[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_206            │ (None, 54, 54, 80)     │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_207 (Conv2D)  

 Total params: 75,704,082 (288.79 MB)

 Trainable params: 10,222,480 (39.00 MB)

 Non-trainable params: 45,036,640 (171.80 MB)

 Optimizer params: 20,444,962 (77.99 MB)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/best_model.keras')

# Function to preprocess image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize (same as training)
    return img_array

# Predict on a new image
img_path = '/content/test_image7.jpg'  # Change to your test image path
img_array = preprocess_image(img_path)
predictions = model.predict(img_array)

# Get class with highest probability
predicted_class = np.argmax(predictions[0])
class_labels = train_generator.class_indices  # Get class labels
class_labels = {v: k for k, v in class_labels.items()}  # Reverse mapping
print(f"Predicted Class: {class_labels[predicted_class]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
Predicted Class: animal_crab
